In [1]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import IPython.display as ipd
import os

In [2]:
#read the audio
song = AudioSegment.from_file('audio.mp4') 

In [3]:
# split into chunks based on prolonged silence
chunks = split_on_silence(song, min_silence_len = 3000, silence_thresh = -32, keep_silence = True)

In [4]:
chunks[0] #listen to some of the chunks to see if the splitting make sence

In [5]:
# save all chunks as audiofiles 
i=0
length_of_chunks = 0
total_length = song.duration_seconds

for audio_chunk in chunks:  
  
  length_of_chunks += audio_chunk.duration_seconds
  name_add = str(i)
  if len(name_add) == 1: 
    name_add = '0' + name_add + '.mp4'
  else: 
    name_add = name_add + '.mp4'
  audio_chunk.export("./chunks/chunk" + name_add, format ="mp4")
  i += 1

In [6]:
# A sanity check: length should be equal since we kept the silent parts
print('Total length: ',total_length )
print('Sum of chunks length: ', length_of_chunks)

Total length:  612.448
Sum of chunks length:  612.4479999999998


In [7]:
ipd.Audio('./chunks/chunk17.mp4') # check that files were saved correctly 

In [3]:
import os
for f in os.listdir('./chunks'):
    os.remove(os.path.join('./chunks/', f))

In [8]:

import torch
import librosa
from datasets import load_dataset
from transformers import (
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
)


In [9]:
#model_name = "facebook/wav2vec2-large-xlsr-53-french"

model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-french"
model = Wav2Vec2ForCTC.from_pretrained(model_name)
processor = Wav2Vec2Processor.from_pretrained(model_name)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:349: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [18]:
import os
list_of_scripts = []
for f in sorted(os.listdir('./chunks')):
    waveform, rate = librosa.load('./chunks/' +f, sr = 16000)
    inputs = processor(waveform, sampling_rate=16_000, return_tensors="pt", padding=True)

    with torch.no_grad():
      logits = model(inputs['input_values'], attention_mask=inputs['attention_mask']).logits

    pred_ids = torch.argmax(logits, dim=-1)
    translation = processor.batch_decode(pred_ids)
    sentence = translation[0][0] + translation[0][1:].lower() +'.'
    list_of_scripts.append(sentence)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1093: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1093: Us

In [19]:
list_of_scripts

["I monsieur le président dominiqe monsi anton mmenskibienvenu à l'assemblée générale ordinaire des actionnaires delbruce sur les comtes de l'exercice arrêter au trente et un décembre deux mille vingt qui sont tenus dans les locaux de l'entreprisedonc aujourd'hui onze octobre deux mille vingt et un sont présents à l'assemblée générale monsieur ernesto kmensky et monsieur dominique hugues.",
 'Oui monsieur kminsky oui voilà plus donc berte.',
 "La présidence de l'assemblée générale et et la déclare urt vingt et une heure quarante.",
 "Donc je constate que laatiemblée général a été convoquée conformément aux statut que cans actions sur saint sont représentées et que l'assemblée peux valablement délibérer sur tous les objets fixés à l'ordre du jour jevai-vous faire l'ecture de l'ors du jour point unes présence point de rapport du conseil dministration relative à l'exercice deux mille vingt point trois examen des compte de résultat et rapport rapport de  organe de rvision point cin approba

In [38]:
from autocorrect import Speller
spell = Speller('fr')

with open('transcript.txt', 'w') as f:
    for item in list_of_scripts:
        spell_corrected = spell(item)
        f.write("%s\n" % spell_corrected)